In [10]:
import os, wandb

## W&B API for CI/CD

The following are examples of operations you can perform with the wandb python client that might be relevant for CI/CD.

Perquisite: the environment variable WANDB_API_KEY

In [52]:
try:
    assert os.getenv('WANDB_API_KEY')
except AssertionError:
    print("WANDB_API_KEY is not set or is empty")

## Getting metrics tied to the current production model

We can get the production model from the registry, then get the metrics via lineage from the associated run.

In [25]:
run = wandb.init(job_type="CI/CD")
path = 'av-team/model-registry/'
model_name = 'BDD Semantic Segmentation'
version = 'production'

artifact = run.use_artifact(f'{path}{model_name}:{version}', 
                            type='model')

Get the run that created the model:

In [27]:
run = artifact.logged_by()

In [28]:
def get_iou_metrics(run):
    return {k:v for k,v in run.summary.items() if '_iou' in k}

get_iou_metrics(run)

{'final_traffic_sign_iou': 0.05314886217585242,
 'bicycle_iou': 0,
 'background_iou': 0.921853196957014,
 'final_road_iou': 0.8347413133027304,
 'traffic_sign_iou': 0.04609753307660045,
 'final_background_iou': 0.920883500388814,
 'vehicle_iou': 0.7598675070628712,
 'person_iou': 0.13954558401437303,
 'final_person_iou': 0.20271520471610835,
 'final_bicycle_iou': 0,
 'final_vehicle_iou': 0.7561426671932289,
 'final_traffic_light_iou': 0.11524698900331648,
 'road_iou': 0.8379052798838932,
 'traffic_light_iou': 0.1546196566812521}

## Getting metrics for a run with a tag

The use case for this would be for comparison purpose such as comparing the model with another run that you have tagged such as "baseline"

In [29]:
api = wandb.Api()
tags = ['candidate']

baseline_runs = api.runs('av-team/mlops-course-001',
                        {"tags": {"$in": tags}}) #this is the Mongo Client

In [30]:
for run in baseline_runs:
    print(run.name)

scarlet-armadillo-502
fine-sweep-63


--- Logging error ---
Traceback (most recent call last):
  File "/Users/ama/mambaforge/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/Users/ama/mambaforge/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "/Users/ama/mambaforge/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/Users/ama/mambaforge/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 328, in _process
    self._sm.send(record)
  File "/Users/ama/mambaforge/lib/python3.10/site-p

--- Logging error ---
Traceback (most recent call last):
  File "/Users/ama/mambaforge/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/Users/ama/mambaforge/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "/Users/ama/mambaforge/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/Users/ama/mambaforge/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 101, in _run
    self._finish()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 331, in _finish
    self._sm.finish()
  File "/Users/ama/mambaforge/lib/python3.10/site-packages/wand

## Promote a model to the registry

This involves moving the model from development stage or experimentation to a centralized repository for access, tracking and management.

In [38]:
path = 'av-team/model-registry/BDD Semantic Segmentation'

In [40]:
run_path = 'av-team/mlops-course-001/2wpuuc70' #run id
api = wandb.Api()
run = api.run(run_path)

wandb: WARNING os.getcwd() no longer exists, using system temp directory


Get the model from the run and promote it to the registry

In [41]:
art = [a for a in run.logged_artifacts() if a.type == 'model']

if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(path, aliases=['new-model-demo'])

CommError: Permission denied to access avri/wanddb_cicd-client/3gqutorv

We can see a model in the registry like this:

In [42]:
versions = api.artifact_versions('model', path)

In [43]:
for v in versions:
    print(f'Model version: {v.version} with tags: {v.aliases}')

Model version: v3 with tags: ['latest', 'production candidate', 'new-model-demo']
Model version: v2 with tags: ['prod candidate', 'demo']
Model version: v1 with tags: ['staging', 'production']
Model version: v0 with tags: []


In [45]:
latest_model = versions[0]

In [46]:
from urllib.parse import urlencode
query = urlencode({'selectionPath': path, 'version': latest_model.version})
registry_url = f'https://wandb.ai/{latest_model.entity}/registry/model?{query}'
registry_url 

'https://wandb.ai/av-team/registry/model?selectionPath=av-team%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v3'

### Run Comparison

In [47]:
import wandb.apis.reports as wr

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [50]:
PROJECT = 'report-wandb-cicd-ex'
ENTITY = 'Ama'

report = wr.Report(
    entity = ENTITY,
    project=PROJECT,
    title='compare Runs',
    description="Demo comparing programatical runs"
)

wandb: WARNING os.getcwd() no longer exists, using system temp directory


In [51]:
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "Run Comparison").set_filters_with_python_expr("Name in ['bountiful-badger-2', 'dastardly-duck-4']")
    ],
    panels=[
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 15}),
    ]
)

report.blocks = report.blocks[:1] + [pg] + report.blocks[1:]
report.save()

wandb: WARNING os.getcwd() no longer exists, using system temp directory
wandb: WARNING os.getcwd() no longer exists, using system temp directory
wandb: WARNING os.getcwd() no longer exists, using system temp directory
wandb: WARNING os.getcwd() no longer exists, using system temp directory


CommError: Permission denied to access avri/wanddb_cicd-client/3gqutorv